In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DecimalType, DoubleType
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .config("fs.s3a.access.key", "Q3AM3UQ867SPQQA43P2F") \
    .config("fs.s3a.secret.key", "zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG") \
    .config("fs.s3a.endpoint", "play.min.io:9000") \
    .appName("VolumeCalculation") \
    .getOrCreate()

sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

In [ ]:
token_transfers_schema = StructType([ \
    StructField("token_address", StringType(), True), \
    StructField("from_address", StringType(), True), \
    StructField("to_address", StringType(), True), \
    StructField("value", DecimalType(38, 0), True), \
    StructField("transaction_hash", StringType(), True), \
    StructField("log_index", LongType(), True), \
    StructField("block_number", LongType(), True), \
  ])

In [ ]:
tokens_schema = StructType([ \
    StructField("address", StringType(), True), \
    StructField("symbol", StringType(), True), \
    StructField("name", StringType(), True), \
    StructField("decimals", LongType(), True), \
    StructField("total_supply", LongType(), True), \
    StructField("block_number", LongType(), True), \
  ])

In [ ]:

basePath = "s3a://bsc-test-tx-volume"

In [ ]:
token_transfers_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(token_transfers_schema) \
    .load(basePath + "/token_transfers/*/*/*.csv")

In [ ]:
tokens_df = spark.read.format("csv") \
    .option("header", True) \
    .schema(tokens_schema) \
    .load(basePath + "/tokens/*.csv")

In [ ]:
token_transfers_df.createOrReplaceTempView("token_transfers_df")
tokens_df.createOrReplaceTempView("tokens_df")

In [ ]:
tokens_result_df = spark.sql("""
SELECT DISTINCT(tt.token_address)
FROM token_transfers_df tt
LEFT JOIN tokens_df t ON LOWER(tt.token_address) = LOWER(t.address)
WHERE t.address IS NULL
""")

In [ ]:
result_df.show(10, False) if result_df.count() > 0 else 0

In [ ]:
import math

start_time = time.time()

result_df.repartition(1) \
    .write \
    .option("header",True) \
    .csv(base_path + "/token_transfer_addresses" + str(math.floor(start_time)))

time.time() - start_time